In [1]:
import json, requests as rq
import pandas as pd

/var/folders/gc/9t_hywd97l7fb1yjstm49l1w0000gn/T/ipykernel_34153/3731561317.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [11]:
rp = rq.get('https://vaccine.meddata.com.ua/index.php?option=com_fabrik&task=meddata.safemed&method=getlist&format=raw&rows=200000')

In [14]:
df = pd.DataFrame(rp.json()['data'])
df['accounting_date'] = pd.to_datetime(df['accounting_date'])
df

,region_order,name,edrpou,trade_name,series,expire_date,expire_date_defrosting,cons_quantity,broken,balance_quantity,accounting_date
0,ІВАНО-ФРАНКІВСЬКА,"КОМУНАЛЬНЕ НЕКОМЕРЦІЙНЕ ПІДПРИЄМСТВО ""ОБЛАСНА ...",1993150,Comirnaty™,SDYX6,2023-06-30,,0,0,225810,2023-06-29
1,ІВАНО-ФРАНКІВСЬКА,"КОМУНАЛЬНЕ НЕКОМЕРЦІЙНЕ ПІДПРИЄМСТВО ""ОБЛАСНА ...",1993150,Comirnaty™,SDYX6,2023-06-30,,0,0,225810,2023-07-13
2,ІВАНО-ФРАНКІВСЬКА,"КОМУНАЛЬНЕ НЕКОМЕРЦІЙНЕ ПІДПРИЄМСТВО ""ОБЛАСНА ...",1993150,Comirnaty™,SDYX6,2023-06-30,,0,0,225810,2023-07-12
3,ІВАНО-ФРАНКІВСЬКА,"КОМУНАЛЬНЕ НЕКОМЕРЦІЙНЕ ПІДПРИЄМСТВО ""ОБЛАСНА ...",1993150,Comirnaty™,SDYX6,2023-06-30,,0,0,225810,2023-07-11
4,ІВАНО-ФРАНКІВСЬКА,"КОМУНАЛЬНЕ НЕКОМЕРЦІЙНЕ ПІДПРИЄМСТВО ""ОБЛАСНА ...",1993150,Comirnaty™,SDYX6,2023-06-30,,0,0,225810,2023-07-10
...,...,...,...,...,...,...,...,...,...,...,...
199995,ДОНЕЦЬКА,"КОМУНАЛЬНЕ НЕКОМЕРЦІЙНЕ ПІДПРИЄМСТВО ""ЦЕНТР ПЕ...",37937247,COMIRNATYTM Tris/Sucrose,GC9425,2024-03-30,,0,0,52800,2023-08-28
199996,ДОНЕЦЬКА,"КОМУНАЛЬНЕ НЕКОМЕРЦІЙНЕ ПІДПРИЄМСТВО ""ЦЕНТР ПЕ...",37937247,COMIRNATYTM Tris/Sucrose,GC9425,2024-03-30,,0,0,52800,2023-08-27
199997,ДОНЕЦЬКА,"КОМУНАЛЬНЕ НЕКОМЕРЦІЙНЕ ПІДПРИЄМСТВО ""ЦЕНТР ПЕ...",37937247,COMIRNATYTM Tris/Sucrose,GC9425,2024-03-30,,0,0,52800,2023-08-26
199998,ДОНЕЦЬКА,"КОМУНАЛЬНЕ НЕКОМЕРЦІЙНЕ ПІДПРИЄМСТВО ""ЦЕНТР ПЕ...",37937247,COMIRNATYTM Tris/Sucrose,GC9425,2024-03-30,,0,0,52800,2023-08-25


In [29]:
grouped_df = df.groupby(['region_order', 'name', 'edrpou', 'trade_name', 'series', 'expire_date']).apply(lambda xs: xs[xs['accounting_date'] == xs['accounting_date'].max()][['cons_quantity','broken','balance_quantity','accounting_date',]])

/var/folders/gc/9t_hywd97l7fb1yjstm49l1w0000gn/T/ipykernel_34153/1121258011.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_df = df.groupby(['region_order', 'name', 'edrpou', 'trade_name', 'series', 'expire_date']).apply(lambda xs: xs[xs['accounting_date'] == xs['accounting_date'].max()][['cons_quantity','broken','balance_quantity','accounting_date',]])


In [30]:
grouped_df

cons_quantity  \
region_order      name                                               edrpou   trade_name                       series expire_date                         
ІВАНО-ФРАНКІВСЬКА ДУ "ТМО МВС України по Івано-Франківській області" 8734204  COMIRNATY™ Original/Omicron BA.1 GD6799 2023-04-30  10538               0   
                  КНП "БУРШТИНСЬКИЙ МІСЬКИЙ ЦЕНТР ПЕРВИННОЇ МЕДИК... 42244782 COMIRNATYTM Tris/Sucrose         GC9425 2024-03-30  39066               0   
                                                                              COMIRNATY™ Original/Omicron BA.1 GD6799 2023-04-30  38368               3   
                                                                              COMIRNATY™ Tris/Sucrose          FY1793 2022-11-30  38427               8   
                                                                                                               FY8211 2022-11-30  38508               0   
...                                                                                                                                                 ...   
ДОНЕЦЬКА          КОМУНАЛЬНЕ ПІДПРИЄМСТВО "ЦЕНТР ПЕРВИННОЇ МЕДИКО... 37803043 COMIRNATY™ Tris/Sucrose          GA2988 2024-02-29  195948              0   
                                                                              Comirnaty™                       SDYX6  2023-06-30  194677             42   
                  КОМУНАЛЬНЕ ПІДПРИЄМСТВО "ЦЕНТР ПЕРВИННОЇ МЕДИКО... 37643758 COMIRNATY™ Tris/Sucrose          GA2988 2024-02-29  192152              0   
                                                                                                                                  192201              0   
                                                                              Comirnaty™                       SDYX6  2023-06-30  191971              0   

                                                                                                                                          broken  \
region_order      name                                               edrpou   trade_name                       series expire_date                  
ІВАНО-ФРАНКІВСЬКА ДУ "ТМО МВС України по Івано-Франківській області" 8734204  COMIRNATY™ Original/Omicron BA.1 GD6799 2023-04-30  10538        0   
                  КНП "БУРШТИНСЬКИЙ МІСЬКИЙ ЦЕНТР ПЕРВИННОЇ МЕДИК... 42244782 COMIRNATYTM Tris/Sucrose         GC9425 2024-03-30  39066        0   
                                                                              COMIRNATY™ Original/Omicron BA.1 GD6799 2023-04-30  38368        5   
                                                                              COMIRNATY™ Tris/Sucrose          FY1793 2022-11-30  38427        4   
                                                                                                               FY8211 2022-11-30  38508        0   
...                                                                                                                                          ...   
ДОНЕЦЬКА          КОМУНАЛЬНЕ ПІДПРИЄМСТВО "ЦЕНТР ПЕРВИННОЇ МЕДИКО... 37803043 COMIRNATY™ Tris/Sucrose          GA2988 2024-02-29  195948       0   
                                                                              Comirnaty™                       SDYX6  2023-06-30  194677       0   
                  КОМУНАЛЬНЕ ПІДПРИЄМСТВО "ЦЕНТР ПЕРВИННОЇ МЕДИКО... 37643758 COMIRNATY™ Tris/Sucrose          GA2988 2024-02-29  192152       0   
                                                                                                                                  192201       0   
                                                                              Comirnaty™                       SDYX6  2023-06-30  191971       4   

                                                                                                                                          balance_quantity  \
region_order      name                 